In [1]:
import pandas
import urllib
import json
import requests
import os
host = 'https://www.encodeproject.org'
experiments = "https://www.encodeproject.org/experiments/"
sample_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_file_manifest_from_website.txt'
control_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_control_file_manifest_from_website.txt'
combined_sample_and_control_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_manifest.txt'
out_dir = '/projects/ps-yeolab3/encode/rnaseq/differential_expression/'
out_dir2 = '/oasis/tscc/scratch/bay001/encode/data/'
to_download = '/home/bay001/projects/encode/permanent_data/to_download_9-14-2016.sh'
to_download2 = '/home/bay001/projects/encode/permanent_data/to_download2_9-14-2016.sh'

### You'll need and manifest file from Eric that has complete imformation on the RBP, Cell Type, CLIP and RNASeq Encode IDs for the experiments. The experiment IDS contain a Accession ID for the whole experiment and then individual ENC IDs for each replicate ( KD and control ). This goes for both CLIP and RNASeq. You'll also need the bam IDs for each replicate

In [2]:
def star_or_tophat(link):
    if os.path.basename(link).find('star') > -1:
        return 'STAR'
    if os.path.basename(link).find('tophat') > -1:
        return 'TOPHAT'
    return 'ambiguous'

In [3]:
X = pandas.read_table('/home/bay001/projects/encode/permanent_data/metadata_from_encodeprojectorg_9-11-2016.tsv')
X.head(2)
print(X.shape)

(6678, 48)


In [4]:
X = X[X['Output type']=='alignments']
X = X[X['Lab']=="ENCODE Processing Pipeline"]

##### Julia found some BAMs that exist on ENCODEPROJECT.ORG but do not show up in my manifest. Are they in Gabe's manifest?
 

In [5]:
X[X['Experiment accession']=='ENCSR031RRO']

,File accession,File format,Output type,Experiment accession,Assay,Biosample term id,Biosample term name,Biosample type,Biosample life stage,Biosample sex,...,md5sum,File download URL,Assembly,Platform,Controlled by,File Status,Audit WARNING,Audit INTERNAL_ACTION,Audit NOT_COMPLIANT,Audit ERROR
5140,ENCFF083QIO,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,68c99b5c960a870136f0b37bade491c3,https://www.encodeproject.org/files/ENCFF083QI...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5141,ENCFF419UWR,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,bf19a65b6c88fe3a09ef9e9f42588148,https://www.encodeproject.org/files/ENCFF419UW...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5142,ENCFF519WJE,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,b0a137276bfa9ce899449e291ec8776c,https://www.encodeproject.org/files/ENCFF519WJ...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5144,ENCFF272FNP,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,6ab2b574554ead07ff80d55f16c18d0c,https://www.encodeproject.org/files/ENCFF272FN...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5146,ENCFF148XKW,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,f7a3450dd9d23f4660fc9c4eec2fd693,https://www.encodeproject.org/files/ENCFF148XK...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5147,ENCFF538ODW,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,d64981ce31dde089bb7c8afe52d8568d,https://www.encodeproject.org/files/ENCFF538OD...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5148,ENCFF840VCF,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,3aedbec0b841bd4f803227e5304191bc,https://www.encodeproject.org/files/ENCFF840VC...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
5150,ENCFF306VXJ,bam,alignments,ENCSR031RRO,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,f258412df622e7c15406784cd57ad269,https://www.encodeproject.org/files/ENCFF306VX...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN


### This block scrolls through the Encode webpage JSON code to extract the correct locations / links to the KD bam files. Links to control bam files are generated further below. For now, names of control bam files are stored in a dictionary, with the accession number as key and the bam file name as value

### This should follow the schema from (as of 9-8-2016): https://github.com/ENCODE-DCC/encoded/blob/master/src/encoded/schemas/file.json

In [6]:
samples = list()
links = list()
controls = list()
error = list()
havenocontrol = list()
i = 0
for h in X['Experiment accession'].dropna():
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        for f in data['files']:
            
            i = i + 1
            # print(data['files'][i]['md5sum'])
            # print(data['files'][i]['content_md5sum'])
            # print( data['files'][i]['replicate']['experiment']['target']['name'])
            if ((host+f['href']).find('ENCFF840VCF') != -1):
                print(f['replicate']['experiment']['target']['name'])
            """if ((host+f['href']).endswith('bam') & 
            (f['output_type'] == u'alignments') &
            (f['lab'][u'name'] == 'encode-processing-pipeline')):
                try:
                    if(i%100==0):
                        print('.'),
                    samples.append({'expt_name':f['replicate']['experiment']['target']['name'],
                                    'expt_cell_line':f['replicate'][u'library']['biosample']['biosample_term_name'],
                                    'biological_replicate_number':f['replicate']['biological_replicate_number'],
                                    'submitted_expt_file_name':f['submitted_file_name'],
                                    'expt_file_name':os.path.basename(f['href']),
                                    'assembly':f['assembly'],
                                    'expt_url':host+f['href'],
                                    'RNASEQ_ControlENC':data['possible_controls'][0]['accession'],
                                    'RNASEQ_ENCODEAccID':h,
                                    'aligner': star_or_tophat(f['submitted_file_name']),
                                    'expt_md5sum': f['md5sum']
                                   })
                    controls.append(data['possible_controls'][0]['accession'])
                except IndexError:
                    havenocontrol.append(h)
                links.append(host+f['href'])
        
samples_df = pandas.DataFrame(samples)
samples_df.to_csv(sample_manifest,
                  sep='\t',
                  index=None)
samples_df"""

Non-specific target control-human
Non-specific target control-human
Non-specific target control-human
Non-specific target control-human
Non-specific target control-human
Non-specific target control-human
Non-specific target control-human
Non-specific target control-human


In [ ]:
! grep ENCFF840VCF $sample_manifest

In [ ]:
# download the gene quantifications

# remove duplicates from this list
havenocontrol = list(set(havenocontrol))
controls = list(set(controls))

In [ ]:
print(set(controls) - set(havenocontrol))
print(set(havenocontrol) - set(controls)) # these rbps don't have controls online.
controls

In [ ]:
samples = list()
error = list()
for h in controls:
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        for i in range(0,len(data['files'])):
            # for key, value in data['files'][i].iteritems():
            #     print(key)
            # print(data['files'][i].iteritems())
            # print( data['files'][i]['replicate']['biological_replicate_number'])
            if ((host+data['files'][i]['href']).endswith('bam') & 
            (data['files'][i]['output_type'] == u'alignments') &
            (data['files'][i]['lab'][u'name'] == 'encode-processing-pipeline')):
                """if(list(X[X['RNASEQ_ControlENC']==h]['RNASEQ_ENCODEAccID'])[0] != data['possible_controls'][0]['accession']):
                    error.append("WARNING: CTRL VS AFF CHANGED: {}:{}:{}".format(
                        h,
                        list(X[X['RNASEQ_ControlENC']==h]['RNASEQ_ENCODEAccID'])[0],
                        data['possible_controls'][0]['accession']))"""
                samples.append({'ctrl_name':data['files'][i]['replicate']['experiment']['target']['name'],
                                'ctrl_cell_line':data['files'][i]['replicate'][u'library']['biosample']['biosample_term_name'],
                                'biological_replicate_number':data['files'][i]['replicate']['biological_replicate_number'],
                                'submitted_ctrl_file_name':data['files'][i]['submitted_file_name'],
                                'ctrl_file_name':os.path.basename(data['files'][i]['href']),
                                'assembly':data['files'][i]['assembly'],
                                'ctrl_url':host+data['files'][i]['href'],
                                'aligner': star_or_tophat(data['files'][i]['submitted_file_name']),
                                'RNASEQ_ControlENC':h,
                                'ctrl_md5sum': data['files'][i]['md5sum']})
                links.append(host+data['files'][i]['href'])
                # controls[data['accession']] = data['possible_controls'][0]['accession']
samples_df = pandas.DataFrame(samples)
samples_df.to_csv(control_manifest,
                  sep='\t',
                  index=None)
samples_df

In [ ]:
import pandas as pd # because i like using pd
import numpy as np

In [ ]:
samples_df = pd.read_table(sample_manifest)
controls_df = pd.read_table(control_manifest)

In [ ]:
samples_df[(samples_df['RNASEQ_ControlENC']=='ENCSR603TCV') & \
           (samples_df['aligner']=='TOPHAT') & \
           (samples_df['assembly']=='hg19') & \
          (samples_df['biological_replicate_number']==1)]

In [ ]:
controls_df.drop_duplicates(inplace=True)
controls_df[(controls_df['RNASEQ_ControlENC']=='ENCSR603TCV') & \
           (controls_df['aligner']=='TOPHAT') & \
           (controls_df['assembly']=='hg19') & \
          (controls_df['biological_replicate_number']==1)]

### perform a pandas merge on the ControlENC, aligner, assembly, and biological replicate number

In [ ]:
merged_df.head(2)

In [ ]:
merged_df = pd.merge(samples_df,controls_df,how='inner',on=['RNASEQ_ControlENC',
                                                            'aligner',
                                                            'assembly',
                                                            'biological_replicate_number',
                                                            ])
merged_df.drop_duplicates(inplace=True)
merged_df[merged_df['ctrl_file_name'].str.contains('ENCFF840VCF')]

In [ ]:
merged_df.to_csv('/home/bay001/projects/encode/permanent_data/rna_seq_manifest_9-14-2016.txt',index=None,sep="\t")

### Skip this step if you don't want to re-generate the download script.

In [ ]:
to_download_file = open(to_download,'w')
to_download_file2 = open(to_download2,'w')
for link in list(set(links)):
    to_download_file.write("wget -O {}{} {};\n".format(out_dir, os.path.basename(link), link))
    to_download_file2.write("wget -O {}{} {};\n".format(out_dir2, os.path.basename(link), link))
to_download_file.close()
to_download_file2.close()
len(list(set(links)))

In [ ]:
! grep ENCFF840VCF $to_download2

In [ ]:
len(links)

In [ ]:
! tail /home/bay001/projects/encode/permanent_data/to_download.sh


In [ ]:
# verify that everything on the manifest looks good:
df = pd.read_table(combined_sample_and_control_manifest)
_test = df[df['expt_name']=='RBFOX2-human']
_testhg19 = _test[_test['assembly']=='hg19']
_testhg19

# Let's check out the file downloaded from ENCODEPROJECT.org

In [ ]:
Y = pd.read_table('/home/bay001/projects/encode/permanent_data/metadata_from_encodeprojectorg_9-11-2016.tsv')
Y = Y[(Y['Output type'] == 'alignments') &
      (Y['Lab'] == 'ENCODE Processing Pipeline')]
Y.shape

In [ ]:
Y.ix[2]

In [ ]:
pd.read_table('/home/bay001/projects/encode/permanent_data/rna_seq_manifest_9-14-2016.txt')

In [ ]:
samples_df[samples_df['expt_file_name']=='ENCFF840VCF.bam']

In [ ]:
controls_df